In [ ]:
import numpy as np
import pandas as pd
import statsmodels.api as sm

# Load data
X_train = pd.read_csv('X_train.csv')
y_train = pd.read_csv('Y_train.csv').squeeze()
X_test = pd.read_csv('X_test.csv')
y_test = pd.read_csv('Y_test.csv').squeeze()

# Ensure X_train and X_test are DataFrames
if not isinstance(X_train, pd.DataFrame):
    X_train = pd.DataFrame(X_train)
if not isinstance(X_test, pd.DataFrame):
    X_test = pd.DataFrame(X_test)

# Add constant term
X_train = sm.add_constant(X_train, has_constant='add')
X_test = sm.add_constant(X_test, has_constant='add')

# Check feature dimensions
print("X_train shape:", X_train.shape)  # Should be (n_samples, 15)
print("X_test shape:", X_test.shape)    # Should be (m_samples, 15)

# Set number of iterations and quantiles
iter_num = 50000
q = [0.2, 0.275, 0.35, 0.425, 0.5, 0.575, 0.65, 0.725, 0.8]

# Train models and make predictions
quan_models = [sm.QuantReg(y_train, X_train).fit(q_i, max_iter=iter_num) for q_i in q]
y_predict = [model.predict(X_test) for model in quan_models]

# Convert prediction results to DataFrame
y_predict_df = pd.DataFrame(np.column_stack(y_predict), columns=[f'q_{q_i:.1f}' for q_i in q])

# Save prediction results to CSV file
y_predict_df.to_csv('prediction_PF.csv', index=False)

print("Prediction results have been saved to prediction_PF.csv")


In [ ]:
import gurobipy as gp
from gurobipy import GRB
import numpy as np
import pandas as pd
import scipy.stats as st
import matplotlib.pyplot as plt
import math
from sklearn.metrics import mean_squared_error, mean_absolute_percentage_error

mpc_bus = [
    [1, 51], [2, 20], [3, 39], [4, 39], [5, 0], [6, 52], [7, 19], [8, 28], [9, 0], [10, 0],
    [11, 70], [12, 47], [13, 34], [14, 14], [15, 90], [16, 25], [17, 11], [18, 60], [19, 45], [20, 18],
    [21, 14], [22, 10], [23, 7], [24, 13], [25, 0], [26, 0], [27, 71], [28, 17], [29, 24], [30, 0],
    [31, 43], [32, 59], [33, 23], [34, 59], [35, 33], [36, 31], [37, 0], [38, 0], [39, 27], [40, 66],
    [41, 37], [42, 96], [43, 18], [44, 16], [45, 53], [46, 28], [47, 34], [48, 20], [49, 87], [50, 17],
    [51, 17], [52, 18], [53, 23], [54, 113], [55, 63], [56, 84], [57, 12], [58, 12], [59, 277], [60, 78],
    [61, 0], [62, 77], [63, 0], [64, 0], [65, 0], [66, 39], [67, 28], [68, 0], [69, 0], [70, 66],
    [71, 0], [72, 12], [73, 6], [74, 68], [75, 47], [76, 68], [77, 61], [78, 71], [79, 39], [80, 130],
    [81, 0], [82, 54], [83, 20], [84, 11], [85, 24], [86, 21], [87, 0], [88, 48], [89, 0], [90, 163],
    [91, 10], [92, 65], [93, 12], [94, 30], [95, 42], [96, 38], [97, 15], [98, 34], [99, 42], [100, 37],
    [101, 22], [102, 5], [103, 23], [104, 38], [105, 31], [106, 43], [107, 50], [108, 2], [109, 8], [110, 39],
    [111, 0], [112, 68], [113, 6], [114, 8], [115, 22], [116, 184], [117, 20], [118, 33]
]
ratio_bus = np.zeros(118)
for i in range(118):
    ratio_bus[i] = mpc_bus[i][1]/sum(mpc_bus[j][1] for j in range(118))
print(ratio_bus)
print(sum(ratio_bus[i] for i in range(118)))
print(sum(mpc_bus[j][1] for j in range(118)))

# Read system parameters
topo = pd.read_excel('118nodes_system.xlsx', sheet_name='topology', index_col=None, header=None)  # Topology structure

unit = pd.read_excel('118nodes_system.xlsx', sheet_name='unit', header=0)  # Generator capacity, price, up/down reserve, up/down reserve price for each node

reserve_up = 150
reserve_down = 150
nodes = 118
test_Y_splits = np.array(pd.read_csv('prediction_PF.csv', header=0))
real_Y_splits = np.array(pd.read_csv('Y_test.csv', header=0))
print(unit.shape[0])
s = test_Y_splits.shape[0]
n = 9
print(real_Y_splits[100][0]*ratio_bus[1])


In [ ]:
# Build the model
model = gp.Model()
P_c = model.addVars(unit.shape[0], s, lb=0, vtype=GRB.CONTINUOUS, name='P_c')  # Thermal unit output
R_U = model.addVars(unit.shape[0], s, lb=0, vtype=GRB.CONTINUOUS, name='R_U')  # Up reserve capacity of thermal units
R_D = model.addVars(unit.shape[0], s, lb=0, vtype=GRB.CONTINUOUS, name='R_D')  # Down reserve capacity of thermal units
r_U = model.addVars(unit.shape[0], s, n, lb=0, vtype=GRB.CONTINUOUS, name='r_U')  # Real-time upward regulation power
r_D = model.addVars(unit.shape[0], s, n, lb=0, vtype=GRB.CONTINUOUS, name='r_D')  # Real-time downward regulation power
P_lsh = model.addVars(topo.shape[0], s, n, lb=0, vtype=GRB.CONTINUOUS, name='P_lsh')  # Load shedding
delta_DA = model.addVars(topo.shape[0], s, n, lb=-180, ub=180, vtype=GRB.CONTINUOUS, name='delta_DA')
delta_RT = model.addVars(topo.shape[0], s, n, lb=-180, ub=180, vtype=GRB.CONTINUOUS, name='delta_RT')
ss = model.addVars(topo.shape[0], s, n, lb=0, vtype=GRB.CONTINUOUS, name='ss')

c = 9000  # Load shedding cost
c_s = 1750

# Set the objective function
model.setObjective(gp.quicksum(R_U[i,j]*unit.iloc[i,5] + R_D[i,j]*unit.iloc[i,6] + P_c[i,j]*unit.iloc[i,2] for i in range(unit.shape[0]) for j in range(s))
                + gp.quicksum((r_U[i,j,l]-r_D[i,j,l])*unit.iloc[i,2] +(P_lsh[i,j,l] * c) + (ss[i,j,l]*c_s) for i in range(topo.shape[0]) for j in range(s) for l in range(n))/n, GRB.MINIMIZE)

for k in range(s):
    # Constraints
        # Reserve demand constraint
    model.addConstr(gp.quicksum(R_U[i,k] for i in range(unit.shape[0])) == reserve_up)
    model.addConstr(gp.quicksum(R_D[i,k] for i in range(unit.shape[0])) == reserve_down)
        # Unit up/down reserve limit
    for i in range(unit.shape[0]):
        model.addConstr(R_U[i,k] <= unit.iloc[i,3])
        model.addConstr(R_D[i,k] <= unit.iloc[i,4])
        # Day-ahead: Power balance at each node
    for l in range(n):
        for t in range(topo.shape[0]):
            model.addConstr(P_c[t,k]-test_Y_splits[k][l]*ratio_bus[t]
            -gp.quicksum(topo.iloc[t,j]*(delta_DA[t,k,l]-delta_DA[j,k,l]) for j in range(topo.shape[0])) >= 0)
        model.addConstr(delta_DA[0,k,l] == 0)
        # Day-ahead: Generation capacity constraint
    for i in range(unit.shape[0]):
        model.addConstr(P_c[i,k] <= unit.iloc[i,1]-R_U[i,k])
        model.addConstr(P_c[i,k] >= R_D[i,k])
        # Real-time: Power balance at each node
    for l in range(n):
        for t in range(topo.shape[0]):
            model.addConstr(P_c[t,k] + r_U[t,k,l] - r_D[t,k,l] - ss[t,k,l] - test_Y_splits[k][l]*ratio_bus[t]  + P_lsh[t,k,l]
            -gp.quicksum(topo.iloc[t,j]*(delta_RT[t,k,l]-delta_RT[j,k,l]) for j in range(topo.shape[0]))==0)
        # Real-time: Reserve capacity constraint
        for i in range(unit.shape[0]):
            model.addConstr(r_U[i,k,l] <= R_U[i,k])
            model.addConstr(r_D[i,k,l] <= R_D[i,k])
        model.addConstr(delta_RT[0,k,l] == 0)
        # Enforce load shedding >= 0
        for t in range(topo.shape[0]):
            model.addConstr(P_lsh[t,k,l] >= 0)  # Ensure load shedding is non-negative
        # Line capacity constraint (day-ahead)
        for i in range(nodes):
            for j in range(nodes):
                if i != j and topo.iloc[i,j] < -0.1:
                    model.addConstr(topo.iloc[i,j] * (delta_DA[i,k,l] - delta_DA[j,k,l]) <= 175)
                    model.addConstr(topo.iloc[i,j] * (delta_DA[i,k,l] - delta_DA[j,k,l]) >= -175)
        # Line capacity constraint (real-time)
        for i in range(nodes):
            for j in range(nodes):
                if i != j and topo.iloc[i,j] < -0.1:
                    model.addConstr(topo.iloc[i,j] * (delta_RT[i,k,l] - delta_RT[j,k,l]) <= 175)
                    model.addConstr(topo.iloc[i,j] * (delta_RT[i,k,l] - delta_RT[j,k,l]) >= -175)

model.optimize()

import numpy as np
import pandas as pd
import gurobipy as gp
from gurobipy import GRB

# Build the second model
model2 = gp.Model()
r_U = model2.addVars(unit.shape[0], s, lb=0, vtype=GRB.CONTINUOUS, name='r_U')  # Real-time upward regulation power
r_D = model2.addVars(unit.shape[0], s, lb=0, vtype=GRB.CONTINUOUS, name='r_D')  # Real-time downward regulation power
P_lsh = model2.addVars(topo.shape[0], s, lb=0, vtype=GRB.CONTINUOUS, name='P_lsh')  # Load shedding
delta_RT = model2.addVars(topo.shape[0], s, lb=-180, ub=180, vtype=GRB.CONTINUOUS, name='delta_RT')
ss = model2.addVars(topo.shape[0], s, lb=0, vtype=GRB.CONTINUOUS, name='ss')
c = 9000  # Load shedding cost
c_s = 1750

# Set the objective function
model2.setObjective(
    gp.quicksum(R_U[i, j].x * unit.iloc[i, 5] + R_D[i, j].x * unit.iloc[i, 6] + P_c[i, j].x * unit.iloc[i, 2] for i in
                range(unit.shape[0]) for j in range(s))
    + gp.quicksum(
        (r_U[i, j] - r_D[i, j]) * unit.iloc[i, 2] + (P_lsh[i, j] * c) + (ss[i, j] * c_s) for i in range(topo.shape[0])
        for j in range(s)),
    GRB.MINIMIZE
)

# Set constraints
for k in range(s):
    # Real-time: Power balance at each node
    for t in range(topo.shape[0]):
        model2.addConstr(
            P_c[t, k].x + r_U[t, k] - r_D[t, k] - ss[t, k] - real_Y_splits[k][0] * ratio_bus[t] + P_lsh[t, k]
            - gp.quicksum(topo.iloc[t, j] * (delta_RT[t, k] - delta_RT[j, k]) for j in range(topo.shape[0])) == 0
        )
    # Real-time: Reserve capacity constraints
    for i in range(unit.shape[0]):
        model2.addConstr(r_U[i, k] <= R_U[i, k].x)
        model2.addConstr(r_D[i, k] <= R_D[i, k].x)
    model2.addConstr(delta_RT[0, k] == 0)
    # Enforce load shedding >= 0
    for t in range(topo.shape[0]):
        model2.addConstr(P_lsh[t, k] >= 0)  # Ensure load shedding is non-negative
    # Line capacity constraint
    for i in range(nodes):
        for j in range(nodes):
            if i != j and topo.iloc[i, j] < -0.1:
                model2.addConstr(topo.iloc[i, j] * (delta_RT[i, k] - delta_RT[j, k]) <= 175)
                model2.addConstr(topo.iloc[i, j] * (delta_RT[i, k] - delta_RT[j, k]) >= -175)

# Optimize model
model2.optimize()

if model2.status == GRB.Status.OPTIMAL:
    # Output the objective value
    objective_value = model2.ObjVal
    print(f"Optimization completed, optimal objective value is: {objective_value}")
